# DS55 — Region × Sector Structural Analysis
**Member Assigned:** MARTIN, Randy Jr.
**Task Name:** Region × Sector Structural Analysis ## Task Description Examine the structural composition of regional economies by comparing sectoral contributions across regions. Classify regions according to whether they are agriculture-heavy, industry-heavy, or service-heavy. ## Expected Outputs - Region × Sector summary table - Classification of regional economic structures - Structural comparison visualization (stacked bar or heatmap) - Interpretation of regional specialization patterns ## Dataset: SECTORAL ECONOMIC STRUCTURE (Sector Level)

# DS55 Region x Sector Structural Analysis
Compares sectoral composition across regions, computes sector shares, classifies each region by dominant sector, and saves structural comparison tables and figures.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Robust BASE_DIR resolution for script and notebook contexts.
try:
    BASE_DIR = Path(__file__).resolve().parents[2]
except NameError:
    cwd = Path.cwd().resolve()
    if (cwd / '03_Cleaned_Data').exists():
        BASE_DIR = cwd
    elif (cwd.parent / '03_Cleaned_Data').exists():
        BASE_DIR = cwd.parent
    else:
        BASE_DIR = cwd.parents[1] if len(cwd.parents) > 1 else cwd

CLEAN_DIR = BASE_DIR / '03_Cleaned_Data'
OUT_TABLES = BASE_DIR / 'outputs' / 'tables'
OUT_FIGS = BASE_DIR / 'outputs' / 'figures'

OUT_TABLES.mkdir(parents=True, exist_ok=True)
OUT_FIGS.mkdir(parents=True, exist_ok=True)

print(f'BASE_DIR: {BASE_DIR}')
print(f'CLEAN_DIR: {CLEAN_DIR}')
print(f'OUT_TABLES: {OUT_TABLES}')
print(f'OUT_FIGS: {OUT_FIGS}')


In [ ]:
# Load Dataset 3 (primary input)
file_d3 = CLEAN_DIR / 'Dataset 3 Region_Sector_Structure (CLEANED).csv'
df = pd.read_csv(file_d3)

print('Shape:', df.shape)
print(df.head())
print('\nUnique regions:', df['Region'].nunique())
print('Unique sectors:', sorted(df['Main_Sector'].dropna().unique().tolist()))
print('Year range:', int(df['Year'].min()), 'to', int(df['Year'].max()))
print('Price types:', sorted(df['Price_Type'].dropna().unique().tolist()))


In [ ]:
# Cleaning / standardization
expected_sectors = [
    'Agriculture, forestry, and fishing',
    'Industry',
    'Services'
]

df['Main_Sector'] = (
    df['Main_Sector']
    .astype(str)
    .str.strip()
    .str.replace(r'\s+', ' ', regex=True)
)

df['Sector_GRDP'] = pd.to_numeric(df['Sector_GRDP'], errors='coerce')

before_rows = len(df)
df = df.dropna(subset=['Sector_GRDP']).copy()
after_numeric_rows = len(df)

# Keep only expected sectors for structural comparison
invalid_sector_rows = (~df['Main_Sector'].isin(expected_sectors)).sum()
df = df[df['Main_Sector'].isin(expected_sectors)].copy()
after_sector_rows = len(df)

print(f'Rows before numeric cleaning: {before_rows}')
print(f'Rows after dropping non-numeric Sector_GRDP: {after_numeric_rows}')
print(f'Rows removed for non-target sectors: {invalid_sector_rows}')
print(f'Rows in final cleaned scope: {after_sector_rows}')
print('Sectors retained:', sorted(df['Main_Sector'].unique().tolist()))


In [ ]:
def safe_tag(value):
    text = str(value).strip()
    text = ''.join(ch if ch.isalnum() else '_' for ch in text)
    while '__' in text:
        text = text.replace('__', '_')
    return text.strip('_')

all_outputs = []
interpretation_rows = []

price_types = sorted(df['Price_Type'].dropna().unique().tolist())
print('Price types to analyze:', price_types)


In [ ]:
# Region x Sector analysis by Price_Type (latest year per price type)
for pt in price_types:
    latest_year = int(df.loc[df['Price_Type'] == pt, 'Year'].max())
    df_scope = df[(df['Price_Type'] == pt) & (df['Year'] == latest_year)].copy()

    print(f'\nAnalyzing Price_Type={pt} | latest_year={latest_year} | rows={len(df_scope)}')

    # Expected Output #1: Region x Sector pivot table
    pivot = pd.pivot_table(
        df_scope,
        index='Region',
        columns='Main_Sector',
        values='Sector_GRDP',
        aggfunc='sum',
        fill_value=0.0
    )

    for sector in expected_sectors:
        if sector not in pivot.columns:
            pivot[sector] = 0.0

    pivot = pivot[expected_sectors].sort_index()
    pivot['Total'] = pivot.sum(axis=1)

    shares = pivot.copy()
    shares['Share_Agri'] = np.where(
        shares['Total'] != 0,
        shares['Agriculture, forestry, and fishing'] / shares['Total'],
        np.nan
    )
    shares['Share_Industry'] = np.where(
        shares['Total'] != 0,
        shares['Industry'] / shares['Total'],
        np.nan
    )
    shares['Share_Services'] = np.where(
        shares['Total'] != 0,
        shares['Services'] / shares['Total'],
        np.nan
    )

    shares_table = shares[['Share_Agri', 'Share_Industry', 'Share_Services']].copy()

    # Expected Output #2: Classification table
    class_df = shares_table.copy().reset_index()
    class_df['Year'] = latest_year
    class_df['Price_Type'] = pt

    share_cols = ['Share_Agri', 'Share_Industry', 'Share_Services']
    share_to_sector = {
        'Share_Agri': 'Agriculture',
        'Share_Industry': 'Industry',
        'Share_Services': 'Services'
    }

    max_col = class_df[share_cols].idxmax(axis=1)
    class_df['Dominant_Sector'] = max_col.map(share_to_sector)
    class_df['Classification'] = class_df['Dominant_Sector'] + '-heavy'
    class_df['Max_Share'] = class_df[share_cols].max(axis=1)

    def second_max(row):
        vals = row[share_cols].to_numpy(dtype=float)
        vals = vals[~np.isnan(vals)]
        if vals.size < 2:
            return np.nan
        return np.sort(vals)[-2]

    class_df['Second_Max_Share'] = class_df.apply(second_max, axis=1)
    class_df['Dominance_Gap'] = class_df['Max_Share'] - class_df['Second_Max_Share']

    class_df = class_df[
        [
            'Region', 'Year', 'Price_Type',
            'Share_Agri', 'Share_Industry', 'Share_Services',
            'Dominant_Sector', 'Classification', 'Max_Share', 'Dominance_Gap'
        ]
    ].sort_values(['Dominant_Sector', 'Max_Share', 'Region'], ascending=[True, False, True]).reset_index(drop=True)

    # Quality check: share sums should be ~1.0
    share_sum = shares_table.sum(axis=1)
    invalid_mask = ~np.isclose(share_sum, 1.0, atol=1e-6, equal_nan=False)
    failing_regions = shares_table.index[invalid_mask].tolist()

    if failing_regions:
        print(f'Share-sum check FAILED for {len(failing_regions)} region(s): {failing_regions}')
    else:
        print('Share-sum check passed for all regions.')

    # Save tables
    tag_pt = safe_tag(pt)
    pivot_path = OUT_TABLES / f'DS55_region_sector_pivot_{tag_pt}_{latest_year}.csv'
    shares_path = OUT_TABLES / f'DS55_region_sector_shares_{tag_pt}_{latest_year}.csv'
    class_path = OUT_TABLES / f'DS55_region_structure_classification_{tag_pt}_{latest_year}.csv'

    pivot.reset_index().to_csv(pivot_path, index=False)
    shares_table.reset_index().to_csv(shares_path, index=False)
    class_df.to_csv(class_path, index=False)

    all_outputs.extend([pivot_path, shares_path, class_path])

    # Expected Output #3A: Stacked bar of shares
    plot_df = shares_table.reset_index().merge(
        pivot[['Total']].reset_index(), on='Region', how='left'
    )
    plot_df['Dominant_Sector'] = class_df.set_index('Region').loc[plot_df['Region'], 'Dominant_Sector'].values
    dominant_order = pd.Categorical(plot_df['Dominant_Sector'], categories=['Agriculture', 'Industry', 'Services'], ordered=True)
    plot_df = plot_df.assign(_dom_order=dominant_order).sort_values(['_dom_order', 'Total'], ascending=[True, False]).drop(columns=['_dom_order'])

    x = np.arange(len(plot_df))
    ag = plot_df['Share_Agri'].to_numpy()
    ind = plot_df['Share_Industry'].to_numpy()
    serv = plot_df['Share_Services'].to_numpy()

    plt.figure(figsize=(14, 6))
    plt.bar(x, ag, label='Agriculture')
    plt.bar(x, ind, bottom=ag, label='Industry')
    plt.bar(x, serv, bottom=ag + ind, label='Services')
    plt.title(f'Region x Sector Shares ({pt}, {latest_year})')
    plt.xlabel('Region')
    plt.ylabel('Share of Regional GRDP')
    plt.xticks(x, plot_df['Region'], rotation=60, ha='right')
    plt.legend()
    plt.tight_layout()

    fig_stack_path = OUT_FIGS / f'DS55_stacked_shares_{tag_pt}_{latest_year}.png'
    plt.savefig(fig_stack_path, dpi=300)
    plt.show()
    all_outputs.append(fig_stack_path)

    # Expected Output #3B: Heatmap of shares (optional, included)
    heat_df = plot_df.set_index('Region')[['Share_Agri', 'Share_Industry', 'Share_Services']]

    plt.figure(figsize=(8, max(6, len(heat_df) * 0.35)))
    im = plt.imshow(heat_df.values, aspect='auto')
    plt.colorbar(im, label='Share')
    plt.title(f'Region x Sector Share Heatmap ({pt}, {latest_year})')
    plt.xlabel('Sector')
    plt.ylabel('Region')
    plt.xticks(np.arange(3), ['Agriculture', 'Industry', 'Services'])
    plt.yticks(np.arange(len(heat_df.index)), heat_df.index)
    plt.tight_layout()

    fig_heat_path = OUT_FIGS / f'DS55_heatmap_shares_{tag_pt}_{latest_year}.png'
    plt.savefig(fig_heat_path, dpi=300)
    plt.show()
    all_outputs.append(fig_heat_path)

    # Expected Output #4: concise interpretation inputs
    dominant_counts = class_df['Dominant_Sector'].value_counts()
    most_common_dom = dominant_counts.idxmax() if not dominant_counts.empty else 'N/A'

    top_serv = class_df.nlargest(3, 'Share_Services')[['Region', 'Share_Services']]
    top_ind = class_df.nlargest(3, 'Share_Industry')[['Region', 'Share_Industry']]
    top_ag = class_df.nlargest(3, 'Share_Agri')[['Region', 'Share_Agri']]

    diversified = class_df.nsmallest(3, 'Dominance_Gap')[['Region', 'Dominance_Gap']]
    specialized = class_df.nlargest(3, 'Dominance_Gap')[['Region', 'Dominance_Gap']]

    interpretation_rows.append({
        'Price_Type': pt,
        'Year': latest_year,
        'Most_Common_Dominant_Sector': most_common_dom,
        'Top3_Service_Heavy': '; '.join([f"{r.Region} ({r.Share_Services:.3f})" for r in top_serv.itertuples(index=False)]),
        'Top3_Industry_Heavy': '; '.join([f"{r.Region} ({r.Share_Industry:.3f})" for r in top_ind.itertuples(index=False)]),
        'Top3_Agriculture_Heavy': '; '.join([f"{r.Region} ({r.Share_Agri:.3f})" for r in top_ag.itertuples(index=False)]),
        'Most_Diversified_3': '; '.join([f"{r.Region} ({r.Dominance_Gap:.3f})" for r in diversified.itertuples(index=False)]),
        'Most_Specialized_3': '; '.join([f"{r.Region} ({r.Dominance_Gap:.3f})" for r in specialized.itertuples(index=False)])
    })

print('\nSaved output files:', len(all_outputs))


In [ ]:
# Interpretation (concise, per Price_Type latest year)
interp_df = pd.DataFrame(interpretation_rows)

for row in interp_df.itertuples(index=False):
    print(f"\nPrice_Type: {row.Price_Type} | Year: {row.Year}")
    print(f"- Sector dominating most regions: {row.Most_Common_Dominant_Sector}")
    print(f"- Top 3 most service-heavy regions: {row.Top3_Service_Heavy}")
    print(f"- Top 3 most industry-heavy regions: {row.Top3_Industry_Heavy}")
    print(f"- Top 3 most agriculture-heavy regions: {row.Top3_Agriculture_Heavy}")
    print(f"- Most diversified regions (small dominance gap): {row.Most_Diversified_3}")
    print(f"- Most specialized regions (large dominance gap): {row.Most_Specialized_3}")


In [ ]:
# Final quality check: ensure all expected outputs exist
missing = [str(p) for p in all_outputs if not Path(p).exists()]

if missing:
    print('Missing outputs detected:')
    for m in missing:
        print('-', m)
else:
    print(f'All outputs exist. Count: {len(all_outputs)}')
